In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# This code is based on https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb

# Analise uma base de código com o Vertex AI Gemini 1.5 Flash


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcode%2Fanalyze_codebase_with_gemini_1_5_pro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Eric Dong](https://github.com/gericdong), [Aakash Gouda](https://github.com/aksstar)|
| | |
|Translate | [Alvaro David](https://github.com/AlvarDev)|

## Visão geral

O Gemini 1.5 Flash apresenta uma janela de contexto longa e inovadora de até 1 milhão de tokens que pode ajudar a analisar, classificar e resumir perfeitamente grandes quantidades de conteúdo em um determinado prompt. Com seu raciocínio de longo contexto, o Gemini 1.5 Flash pode analisar uma base de código inteira para obter insights mais profundos.

Neste tutorial, você aprende como analisar uma base de código inteira com Gemini 1.5 Flash e solicitar ao modelo:

- **Analisar**: Resuma bases de código sem esforço.
- **Guia**: Gere documentação clara de introdução ao desenvolvedor.
- **Depurar**: Descubra bugs críticos e forneça correções.
- **Aprimorar**: implemente novos recursos e melhore a confiabilidade e a segurança.


## Getting Started

### Instalar o SDK da Vertex AI para Python


In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform \
                                        gitpython \
                                        magika

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider 

### Reiniciar o runtime (somente Colab)

Para usar os pacotes recém-instalados neste runtime do Jupyter, você deve reiniciar o runtime. Você pode fazer isso executando a célula abaixo, que reinicia o kernel atual.

A reinicialização pode demorar um minuto ou mais. Depois de reiniciado, continue para a próxima etapa.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel será reiniciado. Aguarde até que termine antes de continuar para a próxima etapa. ⚠️</b>
</div>


### Autentique seu ambiente de notebook (somente Colab)

Se você estiver executando este notebook no Google Colab, execute a célula a seguir para autenticar seu ambiente.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Defina as informações do projeto do Google Cloud e inicialize o SDK do Vertex AI

Para começar a usar a Vertex AI, você precisa ter um projeto existente do Google Cloud e [ativar a API Vertex AI](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Saiba mais sobre [como configurar um projeto e um ambiente de desenvolvimento](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "sre-demos"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importar bibliotecas

In [ ]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
)

## Clonando uma base de código

Você usará o repo [Online Boutique](https://github.com/GoogleCloudPlatform/microservices-demo) como exemplo neste notebook. Online Boutique é um aplicativo de demonstração de microsserviços que prioriza a nuvem. O aplicativo é um aplicativo de comércio eletrônico baseado na web onde os usuários podem navegar pelos itens, adicioná-los ao carrinho e comprá-los. Este aplicativo consiste em 11 microsserviços em múltiplas linguagens de programação.

In [ ]:
# The GitHub repository URL
repo_url = "https://github.com/GoogleCloudPlatform/microservices-demo"  # @param {type:"string"}

# The location to clone the repo
repo_dir = "./repo"

#### Definir funções auxiliares para processar o repositório em GitHub

In [ ]:
import os
import shutil
from pathlib import Path
import requests
import git
import magika

m = magika.Magika()


def clone_repo(repo_url, repo_dir):
    """Clone a GitHub repository."""

    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    os.makedirs(repo_dir)
    git.Repo.clone_from(repo_url, repo_dir)


def extract_code(repo_dir):
    """Create an index, extract content of code/text files."""

    code_index = []
    code_text = ""
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, repo_dir)
            code_index.append(relative_path)

            file_type = m.identify_path(Path(file_path))
            if file_type.output.group in ("text", "code"):
                try:
                    with open(file_path, "r") as f:
                        code_text += f"----- File: {relative_path} -----\n"
                        code_text += f.read()
                        code_text += "\n-------------------------\n"
                except Exception:
                    pass

    return code_index, code_text


def get_github_issue(owner: str, repo: str, issue_number: str) -> str:
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
    }  # Set headers for GitHub API

    # Construct API URL
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"

    try:
        response_git = requests.get(url, headers=headers)
        response_git.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as error:
        print(f"Error fetching issue: {error}")  # Handle potential errors

    issue_data = response_git.json()
    if issue_data:
        return issue_data["body"]
    return ""

#### Crie um índice e extraia o conteúdo de uma base de código

Clone o repositório e crie um índice e extraia o conteúdo dos arquivos de código/texto.

In [ ]:
clone_repo(repo_url, repo_dir)

code_index, code_text = extract_code(repo_dir)

## Analisando a base de código com Gemini 1.5 Flash

Com seu raciocínio de longo contexto, o Gemini 1.5 Flash pode processar a base de código e responder perguntas sobre ela.

#### Carregue o modelo Gemini 1.5 Flash

Saiba mais sobre os [modelos de API Gemini na Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [ ]:
MODEL_ID = "gemini-1.5-flash-001"

model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "Você é um especialista em codificação.",
        """Sua missão é responder a todas as questões
           relacionadas ao código com determinado contexto e instruções.""",
    ],
)

#### Define a helper function to generate a prompt to a code related question

In [ ]:
def get_code_prompt(question):
    """Generates a prompt to a code related question."""

    prompt = f"""
    Questões: {question}

    Contexto:
    - Toda a base de código é fornecida abaixo.
    - Aqui está um índice de todos os arquivos na base de código:
      \n\n{code_index}\n\n.
    - Em seguida, cada um dos arquivos é concatenado.
      Você encontrará todo o código que precisa:
      \n\n{code_text}\n\n

    Resposta:
  """

    return prompt

### 1. Resumindo a base de código


Gere um resumo da base de código.

In [ ]:
question = """
  Dê-me um resumo dessa base de código e diga-me
  as três principais coisas que posso aprender com ela.
"""

prompt = get_code_prompt(question)
contents = [prompt]

# Generate text using non-streaming method
response = model.generate_content(contents)

# Print generated text and usage metadata
IPython.display.Markdown(response.text)
print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')

Este código é um exemplo de aplicação de microserviços chamado "Online Boutique". Ele consiste em 11 microserviços que se comunicam entre si usando gRPC, construídos em diferentes linguagens (Go, Python, C#, Node.js, Java). O frontend é uma aplicação Go que serve o site.

Aqui estão três coisas importantes que você pode aprender com este código:

1. **Microserviços em Ação:** Este exemplo mostra como uma aplicação complexa pode ser decomposta em microserviços independentes e como eles podem ser construídos usando diferentes linguagens e frameworks. Você pode ver a organização do código, a comunicação via gRPC, e a gestão da aplicação como um todo.
2. **Integração com Serviços do Google Cloud:**  O código integra vários serviços do Google Cloud como Google Cloud Build (para construção e push de imagens), Google Kubernetes Engine (para orquestração), Google Cloud Profiler, Google Cloud Trace, Google Cloud Secret Manager. Isso demonstra como microserviços podem ser facilmente integrados com serviços de infraestrutura.
3. **Implementação de um Frontend**:  O frontend da aplicação, feito em Go, é um exemplo de como servir um site com recursos dinâmicos, usando bibliotecas e frameworks populares como o `mux` (para roteamento) e o `template` (para renderização de templates).



Usage metadata:
{'prompt_token_count': 668886, 'candidates_token_count': 282, 'total_token_count': 669168}


### 2. Criação de um guia de primeiros passos para desenvolvedores

Gere um guia de primeiros passos para desenvolvedores. Este exemplo usa a opção de streaming para gerar o conteúdo.

In [ ]:
question = """
  Forneça um guia de primeiros passos para integrar
  novos desenvolvedores à base de código.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)


Aqui estão alguns passos para integrar novos desenvolvedores à base de código:

1. **Configurar o ambiente:** 
    * **Instale as dependências:** Certifique-se de que os novos desenvolvedores tenham todos os softwares, ferramentas e bibliotecas necessárias para trabalhar no projeto (por exemplo, Git, Docker, Skaffold, etc.).
    * **Configure o IDE:** Forneça instruções para configurar o IDE preferido (por exemplo, Visual Studio Code, IntelliJ, etc.) para o projeto, incluindo configurações de formatação de código, depuração e integração de ferramentas.
    * **Crie um ambiente de desenvolvimento local:** Configure um ambiente local para os novos desenvolvedores onde eles possam executar o projeto, como um contêiner Docker ou uma máquina virtual.

2. **Forneça uma visão geral do projeto:**
    * **Explique o propósito do projeto:** Comunique claramente o que o projeto faz, os problemas que ele resolve e seus objetivos.
    * **Apresente a arquitetura do projeto:** Detalhe a estrutura do projeto, os diferentes componentes, como eles interagem e as tecnologias envolvidas.
    * **Mostre a estrutura do código:** Explique os principais diretórios, arquivos e o padrão de nomenclatura usado no projeto.

3. **Guiá-los pela base de código:**
    * **Comece com o ponto de entrada:** Mostre aos novos desenvolvedores o arquivo ou o local onde o projeto é iniciado (por exemplo, o arquivo `main.go`, `index.js`, etc.) e explique como ele funciona.
    * **Explique os principais módulos:** Descreva cada módulo ou componente do projeto, incluindo suas responsabilidades, funções e como eles se conectam.
    * **Demonstre o fluxo do código:** Caminhe pelos desenvolvedores pelas etapas principais do fluxo do código, ilustrando como ele funciona e como as diferentes partes interagem.

4. **Facilite a contribuição:**
    * **Crie tarefas claras e fáceis de entender:** Avise os novos desenvolvedores sobre tarefas apropriadas para iniciantes para que eles comecem a contribuir.
    * **Escreva um guia de estilo de código:** Inclua um guia claro e conciso sobre as convenções de estilo de código do projeto para manter a consistência.
    * **Estabeleça um processo de revisão de código:** Explique o processo de revisão de código, incluindo as ferramentas usadas, as expectativas e o feedback.

5. **Recursos e documentação:**
    * **Documente a API:** Crie uma documentação completa para a API do projeto, incluindo como interagir com os serviços e os dados usados.
    * **Forneça documentação para cada componente:** Crie documentação para cada módulo ou componente, incluindo exemplos de código, melhores práticas e explicações detalhadas.
    * **Utilize um repositório de documentação:** Documente o projeto usando um repositório como `Readme.md` ou documentação específica do projeto, como `docs` ou `wiki`, para facilitar o acesso e a referência.

6. **Comunicação e suporte:**
    * **Defina canais de comunicação:** Crie um canal de comunicação para os novos desenvolvedores se conectarem, fazerem perguntas e compartilharem atualizações.
    * **Forneça suporte contínuo:** Seja acessível para responder a perguntas, fornecer orientação e ajudar os novos desenvolvedores a navegar no projeto.

7. **Usando ferramentas de automação:**
    * **Utilize ferramentas de integração contínua e entrega contínua (CI/CD):** Explique as ferramentas CI/CD utilizadas para o projeto, incluindo como construir, testar e implantar o projeto. 
    * **Configure testes automatizados:** Certifique-se de que os novos desenvolvedores entendam como executar testes automatizados e sua importância no desenvolvimento. 
    * **Documente o processo de construção e implantação:** Escreva instruções detalhadas sobre como construir, testar e implantar o projeto, incluindo comandos e scripts.

Lembre-se de que a integração de novos desenvolvedores é um processo contínuo. Ao oferecer suporte e recursos adequados, você pode ajudar os novos desenvolvedores a se integrarem rapidamente à base de código e a se tornarem membros produtivos da equipe.


### 3. Encontrando bugs

Encontre os três problemas mais graves na base de código.

In [ ]:
question = """
  Encontre os 3 problemas mais graves na base de código.
  Me explique da forma mais clara possível e de uma forma humanamente amigável.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

## Os 3 problemas mais sérios na base de código:

1. **Falta de consistência na implementação de rastreio e métricas:** O rastreio e as métricas estão configurados de forma diferente em cada serviço, causando inconsistência e dificultando a monitorização geral do sistema. Alguns serviços têm um código para habilitar o rastreio, enquanto outros usam o padrão do framework em que foram construídos. Isso complica a análise das informações coletadas e a implementação de uma solução centralizada de monitorização.

2. **Falta de testes de integração:** Embora os testes unitários sejam implementados em vários serviços, a base de código não contém testes de integração, o que significa que a interação entre os microserviços não é totalmente testada. Essa falha pode levar a problemas inesperados em produção e dificultar a detecção de erros relacionados à comunicação inter-serviço.

3. **Implementação desatualizada de Redis:** A maioria dos serviços utiliza Redis para armazenar dados, mas a versão utilizada é desatualizada. Isso pode levar a vulnerabilidades de segurança e problemas de desempenho. É importante atualizar para a última versão disponível para garantir a segurança e o desempenho otimizado do sistema.

## Explicação amigável:

Imagine que você está construindo um grande castelo com diferentes artesãos trabalhando em cada parte. Cada artesão está fazendo um bom trabalho em sua seção, mas não há um plano geral para combinar suas obras. Isso resultará em um castelo desorganizado e difícil de navegar.

Da mesma forma, o código do Online Boutique apresenta algumas falhas:

1. **Monitorização desorganizada:** Cada microserviço está monitorizando o seu próprio progresso, mas não há uma maneira consistente de juntar essas informações. Assim, é difícil ter uma visão geral do funcionamento do sistema.
2. **Falta de teste em conjunto:** Os artesãos estão construindo as suas seções do castelo, mas não estão testando se as seções se encaixam. Isso pode resultar em problemas quando o castelo for construído, e será difícil identificar o problema.
3. **Ferramentas desatualizadas:** A equipe está usando uma ferramenta de construção de tijolos muito antiga. Isso pode levar a tijolos fracos e ao castelo a desmoronar.

É importante corrigir essas falhas para garantir que o Online Boutique seja um sistema estável e fácil de manter.

## Sugestões de correções:

- Implementar uma solução centralizada de rastreio e métricas para garantir a consistência entre os serviços.
- Criar testes de integração para verificar a comunicação entre os microserviços.
- Atualizar a versão do Redis para a última disponível.

Com essas correções, o Online Boutique será um sistema mais robusto e fácil de gerenciar.


### 4. Corrigindo bug

Encontre o problema mais grave na base de código que pode ser corrigido e forneça uma correção de código para ele.

In [ ]:
question = """
  Encontre o bug mais grave na base de código para o qual você pode
  fornecer uma correção de código.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

O bug mais grave em `src/shoppingassistantservice/shoppingassistantservice.py` é a falta de tratamento de erro para o `fetch` na função `talkToGemini`. Se o `fetch` falhar, a função retornará um erro HTTP 500 sem informações específicas sobre o erro, dificultando o debug.

Aqui está a correção:

```python
    @app.route("/", methods=['POST'])
    def talkToGemini():
        print("Beginning RAG call")
        prompt = request.json['message']
        prompt = unquote(prompt)

        # Step 1 – Get a room description from Gemini-vision-pro
        llm_vision = ChatGoogleGenerativeAI(model="gemini-pro-vision")
        message = HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": "You are a professional interior designer, give me a detailed decsription of the style of the room in this image",
                },
                {"type": "image_url", "image_url": request.json['image']},
            ]
        )
        response = llm_vision.invoke([message])
        print("Description step:")
        print(response)
        description_response = response.content

        # Step 2 – Similarity search with the description & user prompt
        vector_search_prompt = f""" This is the user's request: {prompt} Find the most relevant items for that prompt, while matching style of the room described here: {description_response} """
        print(vector_search_prompt)

        docs = vectorstore.similarity_search(vector_search_prompt)
        print(f"Vector search: {description_response}")
        print(f"Retrieved documents: {len(docs)}")
        #Prepare relevant documents for inclusion in final prompt
        relevant_docs = ""
        for doc in docs:
            doc_details = doc.to_json()
            print(f"Adding relevant document to prompt context: {doc_details}")
            relevant_docs += str(doc_details) + ", "

        # Step 3 – Tie it all together by augmenting our call to Gemini-pro
        llm = ChatGoogleGenerativeAI(model="gemini-pro")
        design_prompt = (
            f" You are an interior designer that works for Online Boutique. You are tasked with providing recommendations to a customer on what they should add to a given room from our catalog. This is the description of the room: \n"
            f"{description_response} Here are a list of products that are relevant to it: {relevant_docs} Specifically, this is what the customer has asked for, see if you can accommodate it: {prompt} Start by repeating a brief description of the room's design to the customer, then provide your recommendations. Do your best to pick the most relevant item out of the list of products provided, but if none of them seem relevant, then say that instead of inventing a new product. At the end of the response, add a list of the IDs of the relevant products in the following format for the top 3 results: [<first product ID>], [<second product ID>], [<third product ID>] ")
        print("Final design prompt: ")
        print(design_prompt)
        try:
            design_response = llm.invoke(
                design_prompt
            )
        except Exception as e:
            # Handle exceptions during LLM invocation
            logger.error(f"Exception during LLM invocation: {e}")
            return {'content': f"An error occurred while processing your request. Please try again later."}

        data = {'content': design_response.content}
        return data

    return app

if __name__ == "__main__":
    # Create an instance of flask server when called directly
    app = create_app()
    app.run(host='0.0.0.0', port=8080)
```

Esta correção envolve:

1. **Tratamento de exceções:** Um bloco `try...except` foi adicionado ao redor da chamada `llm.invoke`.
2. **Mensagens de erro personalizadas:** Se uma exceção ocorrer durante a invocação do LLM, uma mensagem de erro informativa é registrada no log e um erro HTTP 500 com uma mensagem personalizada é retornado ao usuário.

Esta correção torna o código mais robusto, melhorando a confiabilidade e a capacidade de depuração do serviço.


### 6. Criando um guia de solução de problemas

Crie um guia de solução de problemas para ajudar a resolver problemas comuns.

In [ ]:
question = """
    Forneça um guia de solução de problemas em português para ajudar
    a resolver problemas comuns.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

## Guia de Solução de Problemas para Online Boutique

Este guia visa auxiliar na resolução de problemas comuns ao desenvolver e executar o Online Boutique, um aplicativo de demonstração de microsserviços do Google Cloud Platform. O guia aborda os principais componentes do aplicativo, as ferramentas usadas para execução local e deploy em diferentes ambientes, e os problemas mais frequentes encontrados durante o desenvolvimento. 

**Antes de começar:**

* Certifique-se de ter instalado os pré-requisitos para a ferramenta que você está usando (Docker, skaffold, kubectl).
* Verifique sua conexão com o cluster do Kubernetes (GKE ou cluster local).
* Verifique as configurações do seu ambiente, como variáveis de ambiente e configurações de rede.

**Problemas comuns:**

**1. Problemas de Compilação e Build:**

* **Erro "No space left on device" no Google Cloud Shell:** Isso ocorre quando o Google Cloud Shell fica sem espaço em disco. Utilize a ferramenta `skaffold` com o perfil `gcb` para construir as imagens no Google Cloud Build: 
    ```bash
    skaffold run -p gcb --default-repo=gcr.io/[PROJECT_ID]
    ```
* **Erros de dependência:** Certifique-se de que você tenha instalado todas as dependências necessárias para o seu ambiente de desenvolvimento, especialmente para serviços em Python e Go. Utilize as ferramentas `pip` e `go mod` para gerenciar as dependências.
* **Erros de compilação específicos para a linguagem:** Consulte a documentação específica de cada linguagem para obter ajuda na resolução de erros de compilação e build.

**2. Problemas de Execução Local:**

* **Erros de conexão de rede:** Certifique-se de que os serviços estejam escutando nas portas corretas e que sua rede local esteja configurada para permitir a comunicação entre os serviços. 
* **Erros de conexão com o banco de dados:** Verifique as credenciais do banco de dados e as configurações de conexão. Certifique-se de que o banco de dados esteja funcionando corretamente e que os serviços tenham permissões adequadas para acessá-lo.
* **Erros de implementação específicos para o serviço:** Consulte a documentação específica de cada serviço para obter ajuda na resolução de erros de implementação.

**3. Problemas de Deploy em Kubernetes:**

* **Erros de deploy:** Verifique os arquivos de manifesto (`kubernetes-manifests`, `kustomization.yaml`) e certifique-se de que eles estejam formatados corretamente. Utilize a ferramenta `kubectl` para depurar erros de deploy. 
* **Erros de imagem:** Certifique-se de que as imagens estejam construídas e empurradas para o registro de contêineres correto. Utilize o comando `kubectl` para verificar o status das imagens e depurar erros de pull.
* **Erros de configuração de serviço:** Verifique as configurações dos serviços (ex: `Service`, `Ingress`) e certifique-se de que estejam configurados corretamente para direcionar o tráfego para os Pods.
* **Erros de configuração de rede:** Verifique as configurações de rede, especialmente se estiver usando `NetworkPolicies` ou `Istio`. Certifique-se de que o tráfego possa fluir entre os serviços e que as regras de rede estejam configuradas corretamente.
* **Problemas de acesso ao serviço:** Certifique-se de que o serviço esteja exposto corretamente (ex: com um `LoadBalancer` ou `Ingress`) e que a URL correta esteja sendo usada para acessá-lo.
* **Erros de estado de Pod:** Verifique o estado dos Pods (ex: `Running`, `Pending`, `Error`) usando o comando `kubectl get pods`. Investigue os logs dos Pods em caso de erros. 

**4. Outros Problemas:**

* **Problemas de integração de serviços:** Verifique a comunicação entre os serviços e certifique-se de que cada serviço esteja respondendo às solicitações do outro. Utilize a ferramenta de depuração apropriada para cada linguagem. 
* **Problemas com a interface do usuário (frontend):** Verifique o código fonte do frontend (em Go) e utilize as ferramentas de desenvolvimento web para depurar erros na interface do usuário.
* **Problemas com o fluxo de trabalho do usuário:** Certifique-se de que o fluxo de trabalho do usuário seja configurado corretamente e que cada microserviço esteja operando como esperado. Utilize ferramentas de depuração de rede para monitorar o fluxo de tráfego e dados.

**Dicas adicionais:**

* **Utilize as ferramentas de monitoramento do Google Cloud Platform:** Utilize ferramentas como o Google Cloud Trace, Google Cloud Monitoring e Google Cloud Profiler para monitorar o desempenho e depurar erros em seus serviços.
* **Utilize logs:** Verifique os logs de cada serviço para obter informações sobre erros. 
* **Utilize ferramentas de depuração:** Utilize ferramentas de depuração apropriadas para cada linguagem (ex: `debugger` para Go, `pdb` para Python) para depurar erros no código fonte.
* **Documentação:** Consulte a documentação do Online Boutique e das ferramentas de desenvolvimento que você está usando (Docker, skaffold, kubectl).
* **Comunidade:** Se você estiver enfrentando dificuldades, procure ajuda na comunidade do Google Cloud Platform ou nas comunidades específicas de cada linguagem.

**Este guia é um ponto de partida para a resolução de problemas. A chave para solucionar problemas é entender a arquitetura do aplicativo e a forma como cada serviço interage com o outro, além de utilizar as ferramentas adequadas para depuração e monitoramento.**


### 7. Tornando o aplicativo mais confiável

Recomende as melhores práticas para tornar o aplicativo mais confiável.


In [ ]:
question = """
  Como posso tornar este aplicativo mais confiável?
  Considere as práticas recomendadas de https://www.r9y.dev/
  Me explique da forma mais clara possível e de maneira amigável.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

Para tornar seu aplicativo mais confiável, você pode considerar as seguintes práticas recomendadas do [r9y.dev](https://www.r9y.dev/):

* **Gerenciamento de dependências:**
    * **Atualize suas dependências regularmente.** O [renovate.json5](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/.github/renovate.json5) já está configurado para automatizar esse processo, mas você pode precisar ajustar as configurações para atender às suas necessidades. Certifique-se de testar todas as atualizações de dependências para garantir que não haja conflitos ou problemas de compatibilidade.
    * **Use ferramentas de gerenciamento de dependências.** O [renovate.json5](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/.github/renovate.json5) já está configurado para automatizar a atualização de dependências, e vários outros arquivos como [requirements.in](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/src/recommendationservice/requirements.in) e [requirements.txt](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/src/recommendationservice/requirements.txt) são usados para gerenciar dependências em diferentes linguagens.
    * **Verifique as dependências para vulnerabilidades.** Você pode usar ferramentas como Snyk ou Dependabot para verificar suas dependências em busca de vulnerabilidades conhecidas.
* **Testes:**
    * **Implemente testes abrangentes.** Certifique-se de cobrir todas as partes essenciais do seu aplicativo com testes, incluindo testes unitários, testes de integração e testes de ponta a ponta.
    * **Automatize seus testes.** Isso garante que os testes sejam executados automaticamente em cada compilação ou antes de cada implantação. Vários workflows de GitHub Action como [ci-pr.yaml](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/.github/workflows/ci-pr.yaml) e [ci-main.yaml](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/.github/workflows/ci-main.yaml) já estão configurados para essa finalidade.
    * **Execute testes em diferentes ambientes.** Execute seus testes em ambientes semelhantes aos seus ambientes de produção para garantir que seu aplicativo funcione como esperado.
* **Implantação:**
    * **Use um pipeline de implantação automatizado.** Isso permite que você implante seu aplicativo de forma rápida e confiável. Você já tem um bom pipeline de implantação automatizado com os workflows de GitHub Action.
    * **Use uma estratégia de implantação de baixo risco.** Isso pode incluir o uso de implantações progressivas, implantações de canário ou testes A/B.
    * **Implante em uma estrutura de múltiplas camadas.** Isso permite que você implante várias versões do seu aplicativo em diferentes camadas, o que pode ser útil para testar novas funcionalidades ou reverter a uma versão anterior se algo der errado.
* **Monitoramento:**
    * **Monitore seu aplicativo de perto.** Isso inclui o monitoramento de indicadores chave de desempenho (KPIs), erros e outros eventos importantes. 
    * **Ative o monitoramento do Google Cloud Operations.** Isso inclui o monitoramento de métricas, registros e rastreamento. Você já tem a configuração do OpenTelemetry Collector com o componente Kustomize [google-cloud-operations](https://github.com/GoogleCloudPlatform/microservices-demo/tree/main/kustomize/components/google-cloud-operations), mas pode precisar configurá-lo para usar o Cloud Trace.
    * **Crie alertas para problemas críticos.** Isso permite que você seja notificado imediatamente se algo der errado com seu aplicativo.
* **Segurança:**
    * **Execute análises de segurança regulares.** Isso inclui a realização de varreduras de vulnerabilidades e revisões de código para encontrar possíveis problemas de segurança. Você já tem a configuração do Snyk com o [renovate.json5](https://github.com/GoogleCloudPlatform/microservices-demo/blob/main/.github/renovate.json5), mas pode precisar ajustar as configurações para atender às suas necessidades.
    * **Use práticas de segurança de codificação.** Isso inclui o uso de práticas recomendadas para proteção contra injeção de SQL, scripts entre sites (XSS) e outras vulnerabilidades comuns.
    * **Criptografe dados sensíveis.** Isso inclui o uso de criptografia para proteger dados como senhas, tokens de acesso e outros dados confidenciais.
* **Outras dicas:**
    * **Use uma estrutura de codificação robusta.** Isso pode incluir o uso de padrões de projeto como microserviços ou programação orientada a objetos.
    * **Documente seu código.** Isso facilita a manutenção do código e ajuda outros desenvolvedores a entender como seu aplicativo funciona.
    * **Compartilhe o conhecimento.** Incentive os membros da equipe a compartilhar o conhecimento, por exemplo, por meio de workshops ou revisões de código.

Ao aplicar essas práticas recomendadas do [r9y.dev](https://www.r9y.dev/), você pode tornar seu aplicativo mais confiável e reduzir o risco de falhas ou problemas de segurança. 


### 8. Tornando o aplicativo mais seguro

Recomende as melhores práticas para tornar o aplicativo mais seguro.

In [ ]:
question = """
  Como você pode proteger o aplicativo?
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

O aplicativo pode ser protegido usando uma variedade de métodos, dependendo dos requisitos específicos de segurança. Aqui estão alguns exemplos:

**Autenticação e Autorização:**

* **Autenticação de Usuário:** Implemente um sistema de login para verificar a identidade dos usuários. Isso pode usar métodos como autenticação baseada em senhas, autenticação de dois fatores ou integração com provedores de identidade de terceiros (como Google, Facebook ou GitHub).
* **Autenticação de API:** Proteja os endpoints da API usando tokens de autenticação (como JWT) ou chaves de API para permitir acesso apenas aos aplicativos autorizados.
* **Controle de Acesso Baseado em Funções (RBAC):** Implemente um sistema RBAC para conceder permissões específicas a diferentes usuários e grupos, garantindo que eles tenham acesso apenas aos recursos autorizados.
* **Autorização de Recursos:** Use autorização de recursos para restringir o acesso aos recursos individuais com base nos papéis ou permissões do usuário.

**Segurança de Dados:**

* **Criptografia:** Criptografe os dados sensíveis em trânsito (usando HTTPS) e em repouso (usando criptografia de dados de banco de dados).
* **Controle de Acesso aos Dados:** Implemente mecanismos para controlar o acesso aos dados sensíveis, como controle de acesso baseado em colunas em bancos de dados.
* **Auditoria:** Grave logs para rastrear o acesso aos dados sensíveis, o que pode ajudar a identificar e investigar atividades maliciosas.
* **Validação de Entrada:** Valide e sanitize as entradas do usuário para evitar injeção de código e outros ataques.

**Segurança de Aplicação:**

* **Gerenciamento de Erros:** Trate os erros adequadamente e evite a divulgação de informações confidenciais em mensagens de erro.
* **Proteção Contra Ataques de XSS (Cross-Site Scripting):** Implemente medidas de proteção para evitar que scripts maliciosos sejam injetados no aplicativo.
* **Proteção Contra Ataques de CSRF (Cross-Site Request Forgery):** Implemente medidas de proteção para evitar que usuários não autenticados executem ações em nome de usuários autenticados.
* **Análise de Segurança Estática (SAST):** Use ferramentas SAST para verificar a vulnerabilidade do código-fonte.
* **Análise de Segurança Dinâmica (DAST):** Use ferramentas DAST para verificar a vulnerabilidade do aplicativo em tempo de execução.

**Infraestrutura de Segurança:**

* **Gerenciamento de Patchs:** Mantenha os sistemas operacionais, as dependências do aplicativo e os softwares de segurança atualizados com os patches mais recentes para evitar vulnerabilidades conhecidas.
* **Firewall:** Configure firewalls para bloquear o acesso não autorizado ao aplicativo.
* **Segurança de Rede:** Implemente medidas de segurança de rede, como VPNs e firewalls, para proteger o aplicativo de ataques de rede.
* **Gerenciamento de Log:** Configure um sistema de gerenciamento de logs para registrar e analisar eventos de segurança, o que pode ajudar a detectar e responder a ataques.

**Segurança de Contêiner:**

* **Imagens de Contêineres Seguras:** Use imagens de contêineres com as melhores práticas de segurança, como a execução como um usuário não privilegiado, a minimização da superfície de ataque e a exclusão de dependências desnecessárias.
* **Segurança de Níveis de Contêineres:** Use ferramentas de segurança de níveis de contêineres (como AppArmor ou SELinux) para restringir as ações que os contêineres podem executar.

**Outros métodos:**

* **Segurança por Design:** Incorpore práticas de segurança em todas as fases do desenvolvimento do aplicativo.
* **Treinamento de Segurança:** Treine sua equipe de desenvolvimento sobre práticas de segurança de aplicativos.
* **Revisão de Segurança:** Conduza revisões de segurança regulares para identificar e corrigir vulnerabilidades.

É importante escolher os métodos de segurança mais adequados para suas necessidades específicas e requisitos de segurança. Além disso, você deve manter seus sistemas e aplicativos atualizados com os patches e as melhores práticas de segurança mais recentes.


### 9. Aprendendo a base de código

Crie um questionário sobre os conceitos usados ​​na base de código.

In [ ]:
question = """
  Crie um questionário sobre os conceitos usados ​​em minha base de código
  para me ajudar a solidificar meu entendimento.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

## Questionário sobre a base de código do Online Boutique

Este questionário ajudará você a solidificar seu entendimento sobre os conceitos usados na base de código do Online Boutique.

**1. Arquitetura de Microsserviços:**

* **a) Descreva a arquitetura geral do Online Boutique.**
* **b) Quais são os microsserviços que compõem o Online Boutique?**
* **c) Quais são as tecnologias/linguagens usadas para implementar cada microsserviço?**
* **d) Explique como os microsserviços se comunicam entre si.**

**2. Contêineres e Orquestração:**

* **a) O que é um contêiner Docker e como ele é usado no Online Boutique?**
* **b) Explique o papel do Skaffold na criação e implementação de contêineres.**
* **c) Quais são os principais comandos do Skaffold para desenvolvimento e implantação?**
* **d) Quais são os principais recursos do Kubernetes usados no Online Boutique?**
* **e) Qual é o propósito do arquivo `kubernetes-manifests/kustomization.yaml`?**
* **f) Explique o conceito de "patches" no Kustomize.**
* **g) Quais são as vantagens de usar o Kustomize?**

**3. Gerenciamento de Configuração:**

* **a) Como as configurações específicas de cada ambiente (local, GKE, AWS, etc.) são gerenciadas no Online Boutique?**
* **b) Explique como as configurações são passadas para os contêineres durante a implantação.**

**4. Testes e CI/CD:**

* **a) Descreva como os testes do Online Boutique são estruturados e executados.**
* **b) Quais são as diferentes etapas do pipeline de CI/CD do Online Boutique?**
* **c) Explique como o GitHub Actions é usado para automatizar o CI/CD.**

**5. Monitoramento e Observabilidade:**

* **a) Como o Online Boutique é monitorado e diagnosticado?**
* **b) Quais ferramentas de monitoramento e observabilidade são usadas?**
* **c) Como o OpenTelemetry é utilizado para rastrear e coletar métricas?**
* **d) Explique como o Google Cloud Operations contribui para o monitoramento e observabilidade do Online Boutique.**

**6. Segurança:**

* **a) Como a segurança é implementada no Online Boutique?**
* **b) Quais são os recursos de segurança do Kubernetes usados?**
* **c) Explique como o Istio é usado para fornecer segurança adicional.**
* **d) Quais são as medidas de segurança aplicadas aos diferentes microsserviços?**

**7. Banco de Dados:**

* **a) Quais são os bancos de dados usados no Online Boutique?**
* **b) Como a persistência de dados é realizada?**
* **c) Explique como a escolha do banco de dados impacta a escalabilidade e o desempenho do sistema.**

**8. Ferramentas e Tecnologias:**

* **a) Quais são as principais ferramentas e tecnologias usadas no desenvolvimento do Online Boutique?**
* **b) Como cada ferramenta contribui para o desenvolvimento e a implantação do aplicativo?**

**9. Variantes de Implantação:**

* **a) Explique o conceito de "componentes" no Kustomize.**
* **b) Quais são os diferentes componentes Kustomize disponíveis para o Online Boutique?**
* **c) Como esses componentes permitem que os usuários personalizem o aplicativo?**
* **d) Quais são as considerações a serem feitas ao combinar diferentes componentes?**

**10. Desafios e Melhorias:**

* **a) Quais são os desafios encontrados no desenvolvimento e implantação de aplicações de microsserviços, como o Online Boutique?**
* **b) Quais são as áreas de potencial melhoria para o Online Boutique?**

Responda a estas perguntas para obter uma compreensão mais profunda do Online Boutique e seus diferentes componentes.


### 10. Criando um tutorial de início rápido

Crie um tutorial de início rápido completo para um componente específico.

In [ ]:
question = """
  Escreva um tutorial de início rápido completo que apresente o AlloyDB,
  mostre como configurá-lo com o CartService e destaque os principais recursos
  do AlloyDB no contexto do aplicativo Online Boutique.
"""

prompt = get_code_prompt(question)
contents = [prompt]

response = model.generate_content(contents)
IPython.display.Markdown(response.text)

```yaml
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization
resources:
- base
components:
- components/alloydb
- components/shopping-assistant
```
This Kustomization file will deploy the Online Boutique with AlloyDB and the Shopping Assistant, allowing you to test the service. 

**How to deploy the Online Boutique with AlloyDB and Shopping Assistant**
1. Make sure you've set up the necessary environment variables (e.g., `PROJECT_ID`, `REGION`, `ALLOYDB_CLUSTER_NAME`, `ALLOYDB_INSTANCE_NAME`, `ALLOYDB_CARTS_DATABASE_NAME`, `ALLOYDB_PRODUCTS_DATABASE_NAME`, `ALLOYDB_PRODUCTS_TABLE_NAME`, `ALLOYDB_USER_GSA_ID`, and `ALLOYDB_SECRET_NAME`). You can find a list of the variables in [kustomize/components/alloydb/README.md](kustomize/components/alloydb/README.md) and the steps to set them up.
2. Make sure you've followed the steps described in [kustomize/components/shopping-assistant/README.md](kustomize/components/shopping-assistant/README.md) to set up the shopping assistant, which include deploying the necessary infrastructure, setting up the database, and configuring the service account.
3. Use the provided [kustomize/components/alloydb/scripts/1_deploy_alloydb_infra.sh](kustomize/components/alloydb/scripts/1_deploy_alloydb_infra.sh) script to deploy the AlloyDB infrastructure. 
4. Use the provided [kustomize/components/shopping-assistant/scripts/2_create_populate_alloydb_tables.sh](kustomize/components/shopping-assistant/scripts/2_create_populate_alloydb_tables.sh) script to populate the database. 
5. Navigate to the `kustomize/` directory in your local repository.
6. Update the `kustomization.yaml` file to include the components you want to use.
7. Run the following command to apply the Kustomize configuration:

```bash
kubectl apply -k .
```

**How to use the Shopping Assistant**

1. After the deployment is complete, navigate to the Online Boutique frontend in your browser.
2. Click the **Assistant** icon in the top right of the website to open the chat window.
3. You can type your request in the chat window or upload an image. 
4. The Shopping Assistant will provide you with relevant products from the catalog based on your request or the uploaded image.

**Key features of AlloyDB in the context of the Online Boutique application:**

- **High Availability and Scalability:** AlloyDB ensures high availability and scalability for the cartservice, which is crucial for handling the influx of user traffic during shopping.
- **Strong Consistency:** AlloyDB's strong consistency model guarantees data accuracy, which is critical for maintaining consistency in the user's shopping cart across multiple requests.
- **Robust Security:** With support for network isolation and encryption, AlloyDB provides robust security for the cartservice, protecting sensitive customer data.
- **Performance Optimization:** AlloyDB's optimized PostgreSQL engine offers enhanced performance for the cartservice, enabling fast retrieval and updates of user carts.

The Shopping Assistant enhances the Online Boutique user experience by:

- **Providing Personalized Recommendations:** The assistant suggests relevant products based on the user's request or the uploaded image, improving the shopping experience and increasing product discovery.
- **Leveraging AI Capabilities:** The Shopping Assistant utilizes powerful AI capabilities, including Natural Language Processing (NLP), to understand the user's intent and provide meaningful responses.
- **Integrating with AlloyDB:** The assistant accesses the product catalog stored in AlloyDB, enabling accurate and up-to-date product recommendations.

By integrating AlloyDB and the Shopping Assistant with the Online Boutique application, you create a more robust, secure, and engaging shopping experience for users.


## Conclusão

Neste tutorial, você aprendeu como usar o Gemini 1.5 Flash para analisar uma base de código e solicitar que o modelo:

- Resuma bases de código sem esforço.
- Gere documentação clara de introdução ao desenvolvedor.
- Descubra bugs críticos e forneça correções.
- Implementar novos recursos e melhorar a confiabilidade e segurança.